This code is to optimize TDA.ipynb file, the main change is using three main function as the filter function:
1. eccentricity
2. PCA
3. KDE

In [ ]:
import os

current_dir = os.getcwd()
parent_dir = os.path.dirname(current_dir)
analyze_path = os.path.join(parent_dir, "utils")

os.chdir(analyze_path)

import ast
import pandas as pd
import numpy as np

import pickle
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from tdam.cover import CubicalCover
from tdam.clustering import FailSafeClustering
from tdam.core_old import MapperAlgorithm
from sklearn.metrics import pairwise_distances
from sklearn.neighbors import KernelDensity
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler, normalize

In [ ]:
import geopandas as gpd
from shapely import wkt

TM2 = 3826

# 原始以 0.001 grid 計算出的區域事故及對應索引, 依照 hex_grid 計算出來的GI
grid_gi_df = pd.read_csv('../ComputedData/Grid/grid_gi.csv')
grid_gi_df['accident_indices'] = grid_gi_df['accident_indices'].apply(ast.literal_eval)
grid_gi_df['geometry'] = grid_gi_df['geometry'].apply(wkt.loads)
grid_gi  = gpd.GeoDataFrame(grid_gi_df, geometry='geometry').set_crs(TM2, allow_override=True)
grid_gi['geometry'] = grid_gi.geometry.centroid

# 熱點屬於哪一個城市
taiwan = gpd.read_file('../Data/OFiles_9e222fea-bafb-4436-9b17-10921abc6ef2/TOWN_MOI_1140318.shp')
taiwan = taiwan[(~taiwan['TOWNNAME'].isin(['旗津區', '頭城鎮', '蘭嶼鄉', '綠島鄉', '琉球鄉'])) & 
                (~taiwan['COUNTYNAME'].isin(['金門縣', '連江縣', '澎湖縣']))].to_crs(TM2)
taiwan_cnty = taiwan[['COUNTYNAME','geometry']].dissolve(by='COUNTYNAME')
taiwan_cnty['geometry'] = taiwan_cnty.buffer(0)

county_join = gpd.sjoin(grid_gi[['geometry']], taiwan_cnty, how='left', predicate='within')
grid_gi['COUNTYNAME'] = county_join['COUNTYNAME']

grid_filter = grid_gi[grid_gi['accident_indices'].str.len() > 0]
grid_filter.reset_index(inplace=True)

In [ ]:
# This is same as all_features but adding hotspot only
all_features_df = pd.read_csv("../ComputedData/ForModel/all_featuresV2.csv")
cols = all_features_df.columns[all_features_df.columns.str.contains('事故位置大類別名稱')] # 高共線
all_features_df.drop(columns=cols, inplace=True)
all_features_df.drop(columns=['hotspot'], inplace=True)

# Mapper
Get filtered_data by running FilterforMapper.py

In [ ]:
filter_full = pd.read_csv("../ComputedData/ForModel/filtered_data.csv")
filter_full.drop(columns=['pc4', 'pc5'], inplace=True)
filter_full

In [ ]:
filter_full = pd.read_csv("../ComputedData/ForModel/filtered_data.csv")
filter_full.drop(columns=['pc4', 'pc5'], inplace=True)

overlaps = [3]
intervals = [10]
detailed_results = []
silhouette_for_intervals = []

for overlap in overlaps:
    for interval in intervals:
        print(f"Processing overlap {overlap}, interval {interval}")
        mapper_algo = MapperAlgorithm(
            cover=CubicalCover(
                n_intervals=interval,
                overlap_frac=overlap / 10
            ),
            clustering=FailSafeClustering(
                KMeans(
                    n_clusters=2,
                    random_state=42
                )
            ),
            n_jobs=-1
        )

        mapper_info = mapper_algo.fit_transform(all_features_df.to_numpy(), filter_full)

        silhouette_for_intervals.append(mapper_info[1])
        result = {
            "overlap": overlap,
            "interval": interval,
            "silhouette": mapper_info[1],
            "mapper_info": mapper_info
        }
        detailed_results.append(result)

        # with open(f"../ComputedData/ForMatrixV2/o{overlap}i{interval}.pkl", 'wb') as file:
        #     pickle.dump(result, file)

detailed_results_df = pd.DataFrame(detailed_results)

In [ ]:
from utils_tda import get_max_categories

col = '道路型態大類別名稱'
all_features_df['最高類別'] = all_features_df.apply(get_max_categories, axis=1)
all_features_df['最高類別'].value_counts()

all_features_df['county'] = grid_filter['COUNTYNAME']
all_features_df['hotspot'] = grid_filter['hotspot']
all_features_df['hotspot'] = all_features_df['hotspot'].apply(lambda x: 'Hotspot' if 'Hotspot' in str(x) else 'Not Hotspot')
all_features_df['facility'] = all_features_df[['youbike_100m_count_mean', 'mrt_100m_count_mean', 'parkinglot_100m_count_mean']].apply(
    lambda row: '1' if (row > 0).any() else '0', axis=1
)
all_features_df['hotspot_facility'] = all_features_df['hotspot'] + '_' + all_features_df['facility']
all_features_df['facility'] = all_features_df['facility'].astype(int) 
all_features_df['county_city'] = all_features_df['county'].apply(lambda x: 'City' if '市' in str(x) else 'County')

all_features_df_speed = pd.read_csv('../ComputedData/ForModel/all_features_df-速限.csv')
all_features_df['original_speed'] = all_features_df_speed['速限-第1當事者_mean']

In [ ]:
all_features_df['bn_feature'] = all_features_df.apply(
    lambda row: 1 if (
        (
        (
            (row['道路型態大類別名稱_單路部分'] > 0) or
            (row['道路型態大類別名稱_其他'] > 0) or
            (row['道路型態大類別名稱_圓環廣場'] > 0) or
            (row['道路型態大類別名稱_平交道'] > 0) or
            (row['道路型態大類別名稱_交岔路'] > 0)
            ) and
        (
            (row['號誌-號誌種類名稱_行車管制號誌(附設行人專用號誌)'] > 0) or
            (row['號誌-號誌種類名稱_行車管制號誌'] > 0) or
            (row['號誌-號誌種類名稱_閃光號誌'] > 0) or
            (row['號誌-號誌種類名稱_無號誌'] > 0)
            ) and
        (
            (row['道路類別-第1當事者-名稱_市區道路'] > 0)
            ) and
        (
            (row['original_speed'] < 50)
            ) and
        (
            (row['facility'] > 0)
            )
        )
    ) else 0,
    axis=1
)

In [ ]:
from collections import Counter
from utils_tda import avg_label, most_common_encoded_label, cond_prob_mixed

# use this for finding specific value ratios in a column
def ratio_in_data(data, col='hotspot', values='Hotspot'):
    """
    choose = 'county_city'
    """
    # 取出要判斷的 Series
    if isinstance(data, pd.DataFrame):
        s = data[col].astype(str)
    else:
        s = pd.Series(data).astype(str)

    # 正規化欲比對的值集合
    if isinstance(values, (list, tuple, set)):
        target = set(map(str, values))
        mask = s.isin(target)
    else:
        mask = (s == str(values))

    return float(mask.mean())

def node_cond_prob(subdf):
    """
    This is conditional probability
    choose = ('hotspot', 'facility')
    choose = ('hotspot', '車道劃分設施-分道設施-路面邊線名稱_無')
    choose = ('hotspot', '號誌-號誌種類名稱_行車管制號誌')
    """

    return cond_prob_mixed(
        subdf=subdf,
        # 有設施的情況下為熱點的機率
        # a_col='hotspot', a_is='Hotspot',
        # b_col='facility', b_rule='>0',
        # 行車管制號誌的情況下為熱點的機率
        # a_col='hotspot', a_is='Hotspot',
        # b_col='號誌-號誌種類名稱_行車管制號誌', b_rule='>0',
        # 無路面邊線的情況下為熱點的機率
        # a_col='hotspot', a_is='Hotspot',
        # b_col='車道劃分設施-分道設施-路面邊線名稱_無', b_rule='>0',
        # 交岔路的情況下為熱點的機率
        # a_col='hotspot', a_is='Hotspot',
        # b_col='道路型態大類別名稱_交岔路', b_rule='>0',
        # 針對bn分析做的圖
        a_col='hotspot', a_is='Hotspot',
        b_col='bn_feature', b_rule='>0',
        alpha=0.5,
        condition="B|A"
    )

# # 用來檢查條件機率效果
# test = all_features_df[['hotspot', '號誌-號誌種類名稱_行車管制號誌']].copy()
# test['號誌-號誌種類名稱_行車管制號誌'] = test['號誌-號誌種類名稱_行車管制號誌'].apply(lambda x: 1 if x > 0.5 else 0)
# test[['hotspot', '號誌-號誌種類名稱_行車管制號誌']].value_counts()

In [ ]:
from TrafficTDApythonUtils.plotsv2 import MapperPlotterSpring

oi = 'o3i10'
choose = 'hotspot'

detailed_results_df = pickle.load(open(f"../ComputedData/ForMatrixV2/{oi}.pkl", "rb"))
mapper_plotter = MapperPlotterSpring(
    detailed_results_df['mapper_info'],
    all_features_df,
    seed=47, iterations=130, dim=2,
    range_lst=[-0.5, 0.5, 0.5, -0.5],
    cmap="Reds",
    encoded_label=ratio_in_data
)
mapper_plotter.create_mapper_plot(choose, avg=True, size_threshold=50, plot_type='spring')
full_info, outliers = mapper_plotter.extract_data()
mapper_plotter.map_colors(threshold=0)
mapper_plotter.plot(set_label=True, size=500, anchor=(0,0),
                    # save_path=f"../ComputedData/ForMatrixV2/Plots/{oi}.png"
                    )

### 這是用來找出哪兩種特徵下熱點集中區有組合

In [ ]:
from TrafficTDApythonUtils.plotsv2 import MapperPlotterSpring

As = ['道路型態大類別名稱_單路部分', '道路型態大類別名稱_交岔路', '道路型態大類別名稱_其他', '道路型態大類別名稱_圓環廣場', '道路型態大類別名稱_平交道']
Bs = ['號誌-號誌種類名稱_無號誌', '號誌-號誌種類名稱_行車管制號誌', '號誌-號誌種類名稱_閃光號誌', '號誌-號誌種類名稱_行車管制號誌(附設行人專用號誌)']

# choose = ('hotspot', 'bn_feature')
choose = 'bn_feature'
overlap = 3
interval = 10
seed = 47

for a in As:
    for b in Bs:

        all_features_df['bn_feature'] = all_features_df.apply(
            lambda row: 1 if (
                (row[a] > 0) and
                (row[b] > 0)
            ) else 0,
            axis=1
        )
        detailed_results_df = pickle.load(open(f"../ComputedData/ForMatrixV2/o{overlap}i{interval}.pkl", "rb"))
        mapper_plotter = MapperPlotterSpring(
            detailed_results_df['mapper_info'],
            all_features_df,
            seed=seed, iterations=130, dim=2,
            range_lst=[-0.5, 0.5, 0.5, -0.5],
            cmap="Greens",
            # encoded_label=node_cond_prob
            encoded_label=ratio_in_data
        )
        mapper_plotter.create_mapper_plot(choose, avg=True, size_threshold=50, plot_type='spring')
        full_info, outliers = mapper_plotter.extract_data()
        mapper_plotter.map_colors(threshold=0)
        mapper_plotter.plot(set_label=True, size=500, anchor=(0,0),
                            save_path=f"../ComputedData/ForMatrixV2/PlotsRatio/o{overlap}i{interval}s{seed}_{choose}_{a}_{b}.png"
                            )

### 這是用來grid search的

In [ ]:
from TrafficTDApythonUtils.plotsv2 import MapperPlotterSpring

overlaps = [3]
intervals = [10]
seeds = [47]
# seeds = [i for i in range(10, 50)]
# choose = ('hotspot', 'bn_feature')
choose = 'hotspot'

for seed in seeds:
    for overlap in overlaps:
        for interval in intervals:
            detailed_results_df = pickle.load(open(f"../ComputedData/ForMatrixV2/o{overlap}i{interval}.pkl", "rb"))
            mapper_plotter = MapperPlotterSpring(
                detailed_results_df['mapper_info'],
                all_features_df,
                seed=seed, iterations=130, dim=2,
                range_lst=[-0.5, 0.5, 0.5, -0.5],
                cmap="Reds",
                # encoded_label=node_cond_prob
                # encoded_label=most_common_encoded_label
                encoded_label=ratio_in_data
                # encoded_label=avg_label
            )
            mapper_plotter.create_mapper_plot(choose, avg=True, size_threshold=50, plot_type='spring')
            full_info, outliers = mapper_plotter.extract_data()
            mapper_plotter.map_colors(threshold=0)
            mapper_plotter.plot(set_label=True, size=500, anchor=(0,0),
                                # save_path=f"../ComputedData/ForMatrixV2/Plots/o{overlap}i{interval}s{seed}_{choose}.png"
                                )